In [1]:
import os
import io
import sys
cwd = os.getcwd()
sys.path.insert(0, cwd)
from module.importData import importData
from module.toWordList import toWordList
from module.steamingWiki import steamingWiki
from module.makeModelGensim import makeModelGensim
from module.toVectore import toVectore
#from module.modelLSTM import modelLSTM
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error, mean_absolute_error, cohen_kappa_score
import pandas as pd
import numpy as np
import gensim
import keras
from keras.models import load_model

Using TensorFlow backend.


In [2]:
np.set_printoptions(threshold=sys.maxsize)
pd.get_option("display.max_rows", 1000)
pd.set_option('display.max_columns', 1000)

In [3]:
wikiSource			= 'idwiki'
answerData			= 'DataAnswerExam_SMP.csv'
questionData		= 'qes.csv'
dirData				= cwd+'/data/'
corpusInput			= wikiSource+'.bz2'
wikiOutput			= wikiSource+'.txt'
fileExtension		= 'bin'
trainingAlgoritm	= 0
numDimension		= 200
modelOutput			= wikiSource+'_word2vec_'+str(numDimension)+'_'+str(trainingAlgoritm)+'.'+fileExtension

In [4]:
dAnswer, dQuestion = importData(answer= dirData+answerData, question= dirData+questionData).openData()

if not(os.path.exists(dirData+modelOutput)):
	if not(os.path.exists(dirData+wikiOutput)):
		steamingWiki(corpusInput=corpusInput, wikiOutput=wikiOutput).execute()
	makeModelGensim(wikiOutput=wikiOutput, modelOutput=modelOutput, numDimension=numDimension, trainingAlgoritm=trainingAlgoritm).execute()

if fileExtension != 'bin':
	model = gensim.models.word2vec.Word2Vec.load(dirData+modelOutput)
else:
	model = gensim.models.KeyedVectors.load_word2vec_format(dirData+modelOutput, unicode_errors='ignore')


In [5]:
crossVal = KFold(n_splits=2, random_state=True, shuffle=True)

count = 1

In [6]:
dAnswer.head()

,Essay_id,Answer,Score,TrueAnswer
0,1,Alumni; ; Kelebihan; Permisalan; Tutorial; Im...,3,lulusan : alumnus; rencana : agenda; keuntunga...
120,1,lulusan : alumnus; rencana : agenda; keuntunga...,5,lulusan : alumnus; rencana : agenda; keuntunga...
365,1,alumni; argument; hasil; example; teknik; imaj...,4,lulusan : alumnus; rencana : agenda; keuntunga...
650,1,alumnus; agenda; laba; sampel; teknik; imajina...,5,lulusan : alumnus; rencana : agenda; keuntunga...
125,1,lulusan : alumnus; rencana : agenda; keuntunga...,5,lulusan : alumnus; rencana : agenda; keuntunga...


In [7]:
xtrain = dAnswer.loc[:,['Answer','TrueAnswer']]

In [8]:
xtrain.head()

,Answer,TrueAnswer
0,Alumni; ; Kelebihan; Permisalan; Tutorial; Im...,lulusan : alumnus; rencana : agenda; keuntunga...
120,lulusan : alumnus; rencana : agenda; keuntunga...,lulusan : alumnus; rencana : agenda; keuntunga...
365,alumni; argument; hasil; example; teknik; imaj...,lulusan : alumnus; rencana : agenda; keuntunga...
650,alumnus; agenda; laba; sampel; teknik; imajina...,lulusan : alumnus; rencana : agenda; keuntunga...
125,lulusan : alumnus; rencana : agenda; keuntunga...,lulusan : alumnus; rencana : agenda; keuntunga...


In [9]:
#keras.utils.to_categorical(dAnswer.loc[:,['Score']], num_classes=6)

In [12]:
class modelLSTM(object):
	from keras.models import Sequential, Model
	from keras.layers import LSTM, Dense, Dropout, Bidirectional, Lambda, Dot, Subtract, Multiply
	import keras.backend as K
	from keras import optimizers

	def biSiamenseModel(self, inputD, distance=False):

		from keras.layers import Input, concatenate, subtract, dot
		from keras import backend as K

		def euclidean_distance(vects):
			x, y = vects
			sum_square = K.sum(K.square(x - y), axis=1, keepdims=True)
			return K.sqrt(K.maximum(sum_square, K.epsilon()))
        
		def cosine_distance(vests):
			x, y = vests
			x = K.l2_normalize(x, axis=-1)
			y = K.l2_normalize(y, axis=-1)
			return -K.mean(x * y, axis=-1, keepdims=True)

		def manhattan_distance(vects):
			x, y = vects
			return K.sum((K.abs(x - y)), axis=1, keepdims=True)

		def exponent_neg_manhattan_distance(vects):
			x, y = vects
			return K.exp(-K.sum(K.abs(x-y), axis=1, keepdims=True))


		MA = Input(shape= inputD, dtype="float32")
		MB = Input(shape= inputD, dtype="float32")

		x = self.Bidirectional(self.LSTM(200, dropout=0.2, recurrent_dropout=0.2, return_sequences=True), input_shape=inputD)(MA)
		x = self.Bidirectional(self.LSTM(150, dropout=0.2, recurrent_dropout=0.2))(x)

		y = self.Bidirectional(self.LSTM(200, dropout=0.2, recurrent_dropout=0.2, return_sequences=True), input_shape=inputD)(MB)
		y = self.Bidirectional(self.LSTM(150, dropout=0.2, recurrent_dropout=0.2))(y)
		# a2 = (self.Lambda(function=lambda a: euclidean_distance(a),output_shape=lambda a: (a,1)))([x,y])

		if distance:
			a2 = (self.Lambda(function=lambda a: cosine_distance(a),output_shape=lambda a: (a,1)))([x,y])
		else:
			#a2 = concatenate([x,y])
			a2 = self.Multiply()([x,y])
		
		out = self.Dense(1, activation='relu')(a2)

		model = self.Model(inputs=[MA, MB], outputs=out)
		sgd = self.optimizers.SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
		model.compile(loss='mean_squared_error', optimizer=sgd, metrics=['mae'])

		model.summary()

		return model	

In [13]:
for dx, dy in crossVal.split(dAnswer):

	trainSAnswer = []
	trainTAnswer = []
	testSAnswer = []
	testTAnswer = []
	
	print("\n--------Fold {}--------\n".format(count))
	train, test= dAnswer.iloc[dx], dAnswer.iloc[dy]
	
	
	xtrain = train.loc[:,['Answer', 'TrueAnswer']]
	xtest = test.loc[:,['Answer', 'TrueAnswer']]
	ytrain = train.loc[:,['Score']].values
	ytest = test.loc[:,['Score']].values
	#ytraincat, idtrain = np.unique(train.loc[:,['Score']].values, return_inverse=True)
	#ytrain = keras.utils.to_categorical(idtrain, len(ytraincat))
	#ytestcat, idtest = np.unique(test.loc[:,['Score']].values, return_inverse=True)
	#ytest = keras.utils.to_categorical(idtest, len(ytestcat))
	#ytrain = keras.utils.to_categorical(train.loc[:,['Score']].values, num_classes=6)
	#ytest = keras.utils.to_categorical(test.loc[:,['Score']].values, num_classes=6)
	
	# bagian ini biar universal aja.
	
	[trainSAnswer.append(toWordList().sentenceToWordList(a1[0], changeNumber2Word= True)) for a1 in xtrain.loc[:,['Answer']].values]
	[trainTAnswer.append(toWordList().sentenceToWordList(a1[0], changeNumber2Word= True)) for a1 in xtrain.loc[:,['TrueAnswer']].values]
	[testSAnswer.append(toWordList().sentenceToWordList(a1[0], changeNumber2Word= True)) for a1 in xtest.loc[:,['Answer']].values]
	[testTAnswer.append(toWordList().sentenceToWordList(a1[0], changeNumber2Word= True)) for a1 in xtest.loc[:,['TrueAnswer']].values]
	
	vtrainSAnswer, vtrainTAnswer = toVectore(essays = trainSAnswer, trueAnswer=trainTAnswer, model = model, numFeature= numDimension, average=False, distance=True).changeToVector()
	vtestSAnswer, vtestTAnswer = toVectore(essays = testSAnswer, trueAnswer=testTAnswer, model = model, numFeature= numDimension, average=False, distance=True).changeToVector()
	modelNetwork = modelLSTM().biSiamenseModel(inputD=(vtrainSAnswer.shape[1], vtrainSAnswer.shape[2]))
	modelNetwork.fit([vtrainSAnswer, vtrainTAnswer], ytrain, batch_size=100, epochs=10)
	pred = modelNetwork.predict([vtestSAnswer, vtestTAnswer])


--------Fold 1--------

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            (None, 200, 200)     0                                            
__________________________________________________________________________________________________
input_4 (InputLayer)            (None, 200, 200)     0                                            
__________________________________________________________________________________________________
bidirectional_5 (Bidirectional) (None, 200, 400)     641600      input_3[0][0]                    
__________________________________________________________________________________________________
bidirectional_7 (Bidirectional) (None, 200, 400)     641600      input_4[0][0]                    
____________________________________________________________________________________

In [14]:
dfa = pd.DataFrame()

In [15]:
pred

array([[4.12373  ],
       [4.105681 ],
       [4.20626  ],
       [4.131884 ],
       [4.1968937],
       [4.171869 ],
       [4.2064395],
       [4.2069206],
       [4.1174836],
       [4.1577287],
       [4.130707 ],
       [4.147804 ],
       [4.1357064],
       [4.180518 ],
       [4.107888 ],
       [4.155219 ],
       [4.2069206],
       [4.1429586],
       [4.169763 ],
       [4.1224422],
       [4.2069206],
       [4.207363 ],
       [4.207067 ],
       [4.1658607],
       [4.214115 ],
       [4.2069206],
       [4.2069206],
       [4.2150407],
       [4.130188 ],
       [4.140612 ],
       [4.2069206],
       [4.2069206],
       [4.2069206],
       [4.189408 ],
       [4.2204037],
       [4.1249185],
       [4.191271 ],
       [4.1904917],
       [4.1928024],
       [4.1676393],
       [4.2069206],
       [4.182911 ],
       [4.2069206],
       [4.127233 ],
       [4.1949053],
       [4.1730113],
       [4.18525  ],
       [4.2069206],
       [4.2122927],
       [4.2069206],


In [16]:
cohen_kappa_score(ytest, np.around(pred), weights='quadratic')

0.0

In [17]:
mean_squared_error(ytest, np.around(pred))

1.4820512820512821

In [18]:
mean_absolute_error(ytest, np.around(pred))

1.117948717948718